In [ ]:
def define_info(text):
    completion = client.chat.completions.create(
      extra_headers={},
      extra_body={},
      model="meta-llama/llama-4-maverick",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": f"""
                        다음은 기업의 채용 공고 상세 내용 원본 텍스트입니다:

                        ========================
                        {text}
                        ========================

                        작업 목표:
                        주어진 원본 텍스트를 읽고 아래 여섯 개 항목을 **반드시 유효한 JSON 객체 하나**로만 출력하세요. 출력 외에 어떤 추가 문장, 설명, 주석도 허용하지 않습니다.

                        요구 출력 형식(엄격):
                        {{
                          "주요업무": [ "<주요업무 항목1>", "<주요업무 항목2>", ... ],
                          "자격요건": [ "<자격요건 항목1>", "<자격요건 항목2>", ... ],
                          "우대사항": [ "<우대사항 항목1>", "<우대사항 항목2>", ... ],
                          "자격요건 키워드": [ "<기술스택/툴/프레임워크1>", "<기술스택2>", ... ],
                          "우대사항 키워드": [ "<기술스택/툴/프레임워크1>", "<기술스택2>", ... ],
                          "직군": "<직군 단일값>"
                        }}

                        출력 규칙(꼭 지키세요):
                        1. 출력은 **오직** 위의 JSON 하나만. 다른 텍스트(설명, 안내문, 예시 등) 절대 넣지 마세요.
                        2. 키 이름은 정확히 "주요업무", "자격요건", "우대사항", "자격요건 키워드", "우대사항 키워드", "직군"으로 사용하세요(대소문자 및 공백까지 동일).
                        3. "주요업무", "자격요건", "우대사항"의 값은 **문자열 배열**이어야 합니다. 각 배열 원소는 원문에서 추출한 핵심 문장(짧은 문장 또는 핵심 구)으로 만드세요. 항목이 여러 개면 각각 별도의 문자열로 나누세요.
                        4. "자격요건 키워드"와 "우대사항 키워드"는 각각 해당 섹션(자격요건, 우대사항)에서 **언급된 기술 스택·프레임워크·언어·툴·클라우드·DB·라이브러리 등**만을 추출해 **간결한 기술명**으로 나열한 문자열 배열이어야 합니다. 예: "Python", "Java", "Spring", "Django", "PyTorch", "TensorFlow", "AWS", "GCP", "MySQL" 등. 소프트스킬(커뮤니케이션, 팀워크 등)이나 경험 연차 등은 포함하지 마세요.
                        5. 키워드 추출 규칙:
                          - 각 키워드는 중복 없이 한 번만 포함하세요(중복 제거).
                          - 가능한 경우 널리 쓰이는 약어/표준명(예: AWS, SQL, React) 형태로 표기하세요. 원문에 특정 표기가 있을 경우 그 표기를 그대로 사용해도 됩니다.
                          - 복합 기술표현(예: "Java + Spring")은 가능한 한 개별 키워드로 분리해서 ["Java","Spring"]으로 만드세요.
                        6. 만약 원문에서 해당 항목(주요업무/자격요건/우대사항)이 명확히 존재하지 않거나 찾을 수 없으면 그 키의 값은 빈 배열 `[]`로 하세요. 키워드 필드도 마찬가지로 기술이 없으면 `[]`로 하세요.
                        7. "직군" 값은 아래 제공된 **채용공고분류 목록** 중에서 **가장 연관있는 하나**를 정확한 항목 이름(띄어쓰기/구두점 동일)으로 출력하세요. **단,** "IT개발ㆍ데이터" 관련 채용공고가 아니라면 `"직군"` 값에 문자열 `"None"`을 반드시 출력하세요(즉 IT개발ㆍ데이터인 경우에만 분류명을 출력하고, 그 외 모든 경우는 "None"으로 표기).
                          채용공고분류 목록:
                          [프론트엔드, 백엔드, 데이터, AI, 네트워크, 보안, 프로젝트관리, 세일즈, 로봇, 게임, QA, 기타]
                        8. JSON 출력은 반드시 **유효한 JSON** (파싱 가능한) 이어야 하며, 모든 문자열은 큰따옴표 `"`로 감싸야 합니다.
                        9. 각 배열 원소(문자열)는 불필요한 기호나 불완전한 문장 대신, 파싱/분석하기 쉬운 **간결한 문장 또는 핵심 구**로 적어주세요(권장 200자 이내).
                        10. 절대 예시(JSON 포함)나 응답 포맷 설명을 결과 출력에 포함하지 마세요 — 오직 결과 JSON만 반환하세요.
                        11. 기술 키워드는 "자격요건"과 "우대사항" 섹션에서 각각 별도로 추출하되 **한 단어로 표현되는 기술명(예: Python, Java, Spring, AWS, MySQL 등)**만 포함하고 "영어 능력 우수"처럼 소프트스킬이나 문장형 표현은 제외하세요; 어떤 기술이 두 섹션에 모두 언급되면 두 배열에 모두 포함하되 각 배열 내에서만 중복을 제거합니다.
                        12. 최종 출력 전에 내부적으로 다음을 확인하세요:
                          - 모든 키가 존재하는가? (존재하지 않으면 빈 배열 또는 "None"으로 채움)
                          - JSON이 파싱 가능한가?
                          - "직군"이 IT개발ㆍ데이터가 아닌 경우 정확히 "None" 문자열을 출력하는가?
                        이제 원문 텍스트 `{text}`를 분석하고 위 규칙을 준수하여 결과 JSON만 출력하세요.
                        """
            }
          ]
        }
      ]
    )
    return(completion.choices[0].message.content)

In [ ]:
def filtering(text):
    completion = client.chat.completions.create(
      extra_headers={},
      extra_body={},
      model="meta-llama/llama-4-maverick",
      messages=[
        {
          "role": "user",
          "content": [
            {
              "type": "text",
              "text": f"""
                        다음은 기업의 채용 공고 제목 텍스트입니다:

                        ========================
                        {text}
                        ========================

                        채용공고 제목만 읽고 "IT개발·데이터" 직군의 채용공고인지 판별해주세요.
                        맞으면 YES, 아니면 NO만 출력해주세요.
                        설명이나 다른 말은 하지 말아주세요.
                        """
            }
          ]
        }
      ]
    )
    return(completion.choices[0].message.content)

In [ ]:
def crawl_job(url):
    res = requests.get(url, headers=headers)
    if res.status_code != 200:
        return None

    soup = BeautifulSoup(res.text, "html.parser")

    script = soup.find_all('script')[0].text
    if('등록된 채용정보가 없습니다' in script):
      return None

    # 마감 확인
    if('style="background: #C7CBD3;box-shadow: none;border-color:#c7cbd3;">마감</a>' in str(soup)):
      return None

    # 공고 이름, 회사명, 위치
    title = soup.select_one("h3.tm_mgt_title").string
    company = soup.select_one("#sidebar > div.tm_main_company_info_wrap.tm_main_common_design01 > div.tm_company_info_short > h2").string

    try:
      place = soup.select("div.col-md-8 > p")[-1].string.strip()
    except:
      place = ''

    career = None
    for tr in soup.select("table.table-striped tr"):
        tds = tr.find_all("td")
        if len(tds) >= 2 and tds[0].get_text(strip=True) == "채용 형태":
            career = tds[1].get_text(strip=True)
            break

    div = soup.select('div.col-md-12')[1]
    job_des = []

    # 줄 단위로 나누고, 빈 줄 제거
    lines = div.get_text(separator="\n").splitlines()
    clean_lines = [line.strip() for line in lines if line.strip()]
    job_des.extend(clean_lines)

    # 최종 문자열로 합치기
    job_des_text = "\n".join(job_des)

    # 상세 내용
    for cnt in range(5):
      try:
        ans = filtering(title)
        if(ans.lower()=='no'): break
        temp = define_info(job_des_text)
        print(1)
        s = temp.strip()
        s = re.sub(r"^```[^\n]*\n", "", s)
        s = re.sub(r"\n```$", "", s)
        data = json.loads(s)
        break
      except:
        continue

    if(cnt==4): print('Something Wrong'); return None

    try:
      return {
          "회사": company if company else None,
          "근무지": place if place else None,
          "고용형태": career,
          "모집명": title if title else None,
          "주요업무": data['주요업무'],
          "자격요건": data['자격요건'],
          "우대사항": data['우대사항'],
          "자격요건 키워드": data['자격요건 키워드'],
          "우대사항 키워드": data['우대사항 키워드'],
          "직군": data['직군'],
          "URL": url
      }
    except:
      return None

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
from openai import OpenAI
import json
import re

headers = {"User-Agent": "Mozilla/5.0"}

# 마지막 공고 번호 찾기
last_job_id = None
for test_id in range(3280930, 0, -1):
    url = f"https://www.theteams.kr/recruit/wanted/{test_id}"
    res = requests.get(url, headers=headers)
    if res.status_code != 200:
        continue

    soup = BeautifulSoup(res.text, "html.parser")
    script = soup.find_all('script')[0].text
    if('등록된 채용정보가 없습니다' in script):
      continue

    last_job_id = test_id
    print(f"마지막 공고 번호 발견: {last_job_id}")
    break
    sleep(0.05)

if last_job_id is None:
    raise ValueError("마지막 공고 번호를 찾지 못했습니다.")

# 마지막 번호부터 1000 개 크롤링
start_id = max(last_job_id - 5, 1)
all_jobs = []

client = OpenAI(
      base_url="https://openrouter.ai/api/v1",
      api_key="KEY",
      )

for job_id in range(start_id, last_job_id + 1):
    url = f"https://www.theteams.kr/recruit/wanted/{job_id}"
    print(f"크롤링 중: {url}")
    job = crawl_job(url)
    if job:
        all_jobs.append(job)
    sleep(0.1)  # 서버 과부하 방지


# ===============================================

# 로컬에 CSV 저장
# df = pd.DataFrame(all_jobs)
# df.to_csv("wanted.csv", index=False, encoding="utf-8-sig")
# print("CSV 저장 완료!")

# ===============================================

In [ ]:
# 드라이브에 CSV 저장
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/the_teams.csv'

df = pd.DataFrame(all_jobs)
df = df[df["직군"]!="None"]
df.to_csv(file_path, index=False, encoding='utf-8-sig')

print("CSV 저장 완료!")